In [24]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.dataset import *
from fastai.structured import *
from fastai.column_data import *

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from math import ceil

from itertools import product
import gc
from dateutil import parser

In [25]:
torch.cuda.set_device(0)

RuntimeError: cuda runtime error (35) : CUDA driver version is insufficient for CUDA runtime version at torch/csrc/cuda/Module.cpp:88

In [26]:
import fastai; fastai.__file__

'/home/jeremy/Documents/repositories/Kaggle/competitive-data-science-predict-future-sales/fastai/__init__.py'

In [27]:
PATH = "data/"
sz=224
arch=resnext101_64
batch_size=5

## Feature Engineering

In [33]:
item_categories = pd.read_csv(f'{PATH}item_categories.csv')
items = pd.read_csv(f'{PATH}items.csv')
shops = pd.read_csv(f'{PATH}shops.csv')
sales = pd.read_csv(f'{PATH}sales_train.csv')
sales_test = pd.read_csv(f'{PATH}test.csv')

In [34]:
index_cols  = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'},'item_price': np.mean})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
# all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
all_data = pd.merge(grid,gb,how='left',on=index_cols)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

# del grid,cur_shops,cur_items,gb
# del sales
gc.collect()

/home/jeremy/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


183

In [37]:
all_data.head()

,shop_id,item_id,date_block_num,target,mean
139255,0,19,0,NaN,NaN
141495,0,27,0,NaN,NaN
144968,0,28,0,NaN,NaN
142661,0,29,0,NaN,NaN
138947,0,32,0,6.0,221.0


In [38]:
gb.head()

,shop_id,item_id,date_block_num,target,mean
0,0,30,1,31.0,265.0
1,0,31,1,11.0,434.0
2,0,32,0,6.0,221.0
3,0,32,1,10.0,221.0
4,0,33,0,3.0,347.0


In [30]:
all_data.columns

Index(['shop_id', 'item_id', 'date_block_num', 'target', 'mean'], dtype='object')

In [39]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

all_data.fillna(0.0,inplace=True)
all_data['target'] = all_data['target'].clip(0.0,40.0)
all_data['month'] = all_data['date_block_num'] % 12
all_data['year'] = all_data['date_block_num'] // 12

In [40]:
sales_test['date_block_num'] = 34
sales_test['month'] = sales_test['date_block_num']%12
sales_test['year'] = sales_test['date_block_num']//12

all_data = pd.concat([all_data,sales_test])

# del sales_test
gc.collect()

50

In [43]:
all_data = all_data.merge(items, on="item_id")
all_data.drop('item_name',axis = 1,inplace=True)
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

all_data = downcast_dtypes(all_data)

del items
gc.collect()

87

In [45]:
lag_variable = ['target']
lags = [1 ,2 ,3 ,4, 5, 12]
for lag in lags:
    sales_new_df = all_data[['date_block_num','shop_id','item_id']+lag_variable].copy()
    sales_new_df.date_block_num+=lag
    sales_new_df.columns = ['date_block_num','shop_id','item_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variable]
    all_data = pd.merge(all_data, sales_new_df,on=['date_block_num','shop_id','item_id'] ,how='left')
    del sales_new_df
    gc.collect()

for feat in all_data.columns:
    if 'target' in feat:
        all_data[feat]=all_data[feat].fillna(0)

In [46]:

all_data['avg_sales_per_shop_id'] = 0.0
all_data['avg_sales_per_item_id'] = 0.0
all_data['avg_sales_per_item_cat_id'] = 0.0

avg_shop_sales_per_month = all_data.groupby(['shop_id','date_block_num'],as_index=False).agg({'target':'mean'})
avg_item_sales_per_month = all_data.groupby(['item_id','date_block_num'],as_index=False).agg({'target':'mean'})
avg_item_cat_sales_per_month = all_data.groupby(['item_category_id','date_block_num', ],as_index=False).agg({'target':'mean'})

#lets calculate  6 lags
lag_variable = ['item_id','shop_id','item_category_id']
lags = [1 ,2 ,3 ,4, 5, 12]
curr = 0
for lag in lags:
    diff = lag - curr
    curr = lag
    avg_shop_sales_per_month.date_block_num+=diff
    avg_item_cat_sales_per_month.date_block_num+=diff
    avg_item_sales_per_month.date_block_num+=diff
    
    avg_shop_sales_per_month.columns = ['shop_id','date_block_num']+ [lag_variable[1]+'_lag_'+str(lag)]
    avg_item_sales_per_month.columns = ['item_id','date_block_num']+ [lag_variable[0]+'_lag_'+str(lag)]
    avg_item_cat_sales_per_month.columns = ['item_category_id','date_block_num']+ [lag_variable[2]+'_lag_'+str(lag)]
    
    all_data = pd.merge(all_data, avg_shop_sales_per_month,on=['date_block_num','shop_id'] ,how='left')
    all_data = pd.merge(all_data, avg_item_sales_per_month,on=['date_block_num','item_id'] ,how='left')
    all_data = pd.merge(all_data, avg_item_cat_sales_per_month,on=['date_block_num','item_category_id'] ,how='left')
    
    gc.collect()
del avg_item_cat_sales_per_month,avg_item_sales_per_month,avg_shop_sales_per_month

In [ ]:
all_data.drop(columns=['mean'],inplace=True)
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [ ]:
all_data.fillna(0,inplace=True)
gc.collect()

In [ ]:
all_data.to_pickle(PATH + 'all_data.pkl')

In [ ]:
all_data = pd.read_pickle(PATH + 'all_data.pkl')

In [ ]:
all_data.columns

### Adding category names

In [ ]:
l_cat = list(item_categories.item_category_name)
for ind in range(0,1):
    l_cat[ind] = 'PC Headsets / Headphones'
for ind in range(1,8):
    l_cat[ind] = 'Access'
l_cat[8] = 'Tickets (figure)'
l_cat[9] = 'Delivery of goods'
for ind in range(10,18):
    l_cat[ind] = 'Consoles'
for ind in range(18,25):
    l_cat[ind] = 'Consoles Games'
l_cat[25] = 'Accessories for games'
for ind in range(26,28):
    l_cat[ind] = 'phone games'
for ind in range(28,32):
    l_cat[ind] = 'CD games'
for ind in range(32,37):
    l_cat[ind] = 'Card'
for ind in range(37,43):
    l_cat[ind] = 'Movie'
for ind in range(43,55):
    l_cat[ind] = 'Books'
for ind in range(55,61):
    l_cat[ind] = 'Music'
for ind in range(61,73):
    l_cat[ind] = 'Gifts'
for ind in range(73,79):
    l_cat[ind] = 'Soft'
for ind in range(79,81):
    l_cat[ind] = 'Office'
for ind in range(81,83):
    l_cat[ind] = 'Clean'
l_cat[83] = 'Elements of a food'

In [ ]:
all_data.item_category_id.apply(lambda x: print(type(x))

In [ ]:
all_data['item_category_name'] = all_data.item_category_id.apply(
    lambda x: l_cat[x]
)

In [ ]:
cat_list = list(all_data.item_category_name)

In [ ]:
cat_vars = [
    'date_block_num',
    'item_id',
    'month',
    'shop_id',
    'year',
    'item_category_id',
    'item_category_name'
]

dep = 'target'

In [ ]:
all_data.columns

In [ ]:
for cat in cat_vars:
    all_data[cat] = all_data[cat].astype('category').cat.as_ordered()

In [ ]:
X_train = all_data[(all_data['date_block_num']>12)&(all_data['date_block_num']<33)].drop(['target','ID'], axis = 1)
y_train = all_data[(all_data['date_block_num']>12)&(all_data['date_block_num']<33)]['target']
X_val =  all_data[all_data['date_block_num']==33].drop(['target','ID'], axis = 1)
y_val = all_data[all_data['date_block_num'] == 33]['target']
X_test = all_data[(all_data['date_block_num']==34)].drop(['target'],axis=1)
del all_data

In [ ]:
X_test = X_test.drop('ID', axis=1)

In [ ]:
gc.collect()

In [ ]:
X_train.to_pickle(PATH + 'X_train.pkl')
y_train.to_pickle(PATH + 'y_train.pkl')
X_val.to_pickle(PATH + 'X_val.pkl')
y_val.to_pickle(PATH + 'y_val.pkl')
X_test.to_pickle(PATH + 'X_test.pkl')

In [ ]:
X_train.head().columns

In [ ]:
orig_len = len(X_test)
X_test = X_test.merge(sales_test, on=['item_id', 'shop_id'], how='inner')
X_test = X_test.set_index('ID').sort_index().reset_index().drop('ID', axis=1)
assert len(X_test) == orig_len  # sanity check

In [ ]:
X_train.head()

In [ ]:
X_test.head()

# Training

## _Deep Learning Approach_

In [ ]:
X_train = pd.read_pickle(PATH + 'X_train.pkl')
y_train = pd.read_pickle(PATH + 'y_train.pkl')
X_val = pd.read_pickle(PATH + 'X_val.pkl')
y_val = pd.read_pickle(PATH + 'y_val.pkl')
X_test = pd.read_pickle(PATH + 'X_test.pkl')

In [ ]:
def rmse(y_pred, targ):
    return math.sqrt(((targ - y_pred)**2).mean())

In [ ]:
X_train['target'] = y_train
X_val['target'] = y_val
X_test['target'] = 0

In [ ]:
df, y, nas, mapper = proc_df(X_train, 'target', do_scale=True)
val_df, val_y, nas, mapper = proc_df(
    X_val,
    dep,
    do_scale=True,
    mapper=mapper,
)
test_df, _, nas, mapper = proc_df(
    X_test,
    dep,
    do_scale=True,
    mapper=mapper,
)

In [ ]:
test_df.columns

In [ ]:
y_train = (y.clip(0.,40.))
y_val = (val_y.clip(0.,40.))

In [ ]:
md = ColumnarModelData.from_data_frames(
    path=PATH,
    trn_df=df,
    trn_y=y_train,
    val_df=val_df,
    val_y=y_val,
    cat_flds=cat_vars,
    bs=256,
    test_df=test_df,
    is_reg=True,  # is regression
    is_multi=False, 
)

In [ ]:
cat_sz = [(c, len(X_train[c].cat.categories)+1) for c in cat_vars]

In [ ]:
cat_sz

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [ ]:
emb_szs

In [ ]:
m = md.get_learner(
    emb_szs=emb_szs,
    n_cont=len(df.columns) - len(cat_vars),
    emb_drop=.01,
    out_sz=1,
    szs=[1000, 500, 500],
    drops=[.001, .005, .005],
    use_bn=True,
)

In [ ]:
m.crit = nn.MSELoss()

In [ ]:
m.lr_find()

In [ ]:
m.sched.plot()

In [ ]:
lr = 1e-5
m.fit(lrs=lr, n_cycle=2, metrics=[rmse], cycle_len=1, cycle_mult=2)

### Random Forest Regressor

In [ ]:
val_y

In [ ]:
y

In [ ]:
rf_m = RandomForestRegressor(n_jobs=-1)

In [ ]:
rf_m.fit(df, y)

In [ ]:
result = rf_m.predict(val_df)
rmse(result, val_y)

In [ ]:
rf_result = rf_m.predict(test_df)

In [ ]:
np.unique(rf_result)

In [ ]:
predictions

## Gradient Boost Regressor

In [ ]:
sales_test.head()

In [ ]:
X_test.head()

In [ ]:
asdf = X_test.merge(sales_test, on=['item_id', 'shop_id'], how='inner')

In [ ]:
len(asdf)

In [ ]:
test_asdf = asdf.set_index('ID').sort_index().reset_index().drop('ID', axis=1)

In [ ]:
for var in cat_vars:
    X_train[var] = X_train[var].astype('int32')
    X_val[var] = X_val[var].astype('int32')
    X_test[var] = X_test[var].astype('int32')

In [ ]:
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
params = {
    'objective': 'reg:linear',
    'max_depth':4 ,
    'learning_rate': 1,
    'silent': 1.0,
    'n_estimators': 30,
    'min_child_weight':10
}

bst = XGBRegressor(**params).fit(X_train, y_train,eval_metric='rmse')
preds = bst.predict(X_val)
sqrt(mean_squared_error(y_val, preds))

In [ ]:
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
params = {
    'objective': 'reg:linear',
    'max_depth':4 ,
    'learning_rate': 1,
    'silent': 1.0,
    'n_estimators': 30,
    'min_child_weight':10
}

X = pd.concat([X_train,X_val])
y = np.concatenate([y_train,y_val])
del X_train,X_val,y_train,y_val
gc.collect()

In [ ]:
bst = XGBRegressor(**params).fit(X, y,eval_metric='rmse')

In [ ]:
preds = bst.predict(X_test)
preds = np.clip(preds,0.0,20.0)

In [ ]:
preds = bst.predict(test_asdf)
preds = np.clip(preds, 0.0,20.0)

In [ ]:
preds

## Submit It!

In [ ]:
result = m.predict(is_test=True)
# predictions = np.expm1(result).clip(0, 20)
predictions = result.clip(0,20)
display(predictions)

In [ ]:
final_dataframe = pd.read_csv(f'{PATH}test.csv')

In [ ]:
final_dataframe['item_cnt_month'] = predictions
final_dataframe = final_dataframe[['ID', 'item_cnt_month']]

In [ ]:
final_dataframe.head()

In [ ]:
len(final_dataframe)

In [ ]:
len(np.unique(final_dataframe['ID']))

In [ ]:
SUBM = 'sub/'
os.makedirs(SUBM, exist_ok=True)
final_dataframe.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [ ]:
FileLink(f'{SUBM}subm.gz')